In [ ]:
# Imports
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import MinMaxScaler, StandardScaler


In [ ]:
# Read in the data
college_data = pd.read_csv("https://github.com/UVADS/DS-3021/raw/main/data/cc_institution_details.csv")
job_data = pd.read_csv("https://raw.githubusercontent.com/DG1606/CMS-R-2020/master/Placement_Data_Full_Class.csv")

### Dataset 1, Step 1

In [ ]:
college_data.info()

#### Problems: 
This dataset shares information about many colleges, and there are many columns included. We have metrics on class size, graduation percentage, financial information, test scores, and many others. However, a lot of the data is missing. Students may care about how much money they can receive in awards if they are interested in completing research,, but they may not have access to this information. The problem is students not being able to see much and how commonly award money is given and needing to see what information can show how much money schools award.

#### Question:
Can we predict how much money schools spend on awards from public information?

### Dataset 1, Step 2
- Our independent business metric is the amount of money that schools spend on awards as this is what we are looking to forecast.
- Taking a look at what information we can easily acquire, we can start with level, private vs. public, student count, median SAT score, graduation percentage, and % of full time students for our predictors. Our target variable is money spent on awards, so we will look at the awards_per_value and exp_award_value, the amount of awards given out per 100 undergraduates.

### Data Preparation

In [ ]:
# Creating a new dataset that includes only the variables of interest
college_data_filtered = college_data[['chronname', 'level', 'control', 'student_count', 'med_sat_value', 'grad_100_percentile',
                                       'ft_pct', 'exp_award_value', 'awards_per_value']]


In [ ]:
# Checking structure of our dataset to see if there are any issues
college_data_filtered.info()

Let us explore how many missing values we have: 

In [ ]:
print(college_data_filtered.isnull().sum())

It looks like we are missing a lot of median SAT value data. There is so much missing data that it seems we are better off dropping it entirely. For the grad_100_percentile, we are missing 300 values. This is a small enough subset of our data that we can drop rows missing grad_100_percentile(and the four missing ft_pct) and not hurt model performance. 

In [ ]:
college_data_filtered = college_data_filtered[['chronname', 'level', 'control', 'student_count', 'grad_100_percentile',
                                       'ft_pct', 'exp_award_value', 'awards_per_value']]
college_data_filtered = college_data_filtered.dropna(subset=['grad_100_percentile', 'ft_pct'])

In [ ]:
# Checking how many unique values we have for certain variables that may be categorical
print("Level classifications:", college_data_filtered['level'].unique())
print("Control classifications:", college_data_filtered['control'].unique())

Great. There are only 4-year and 2-year universities, and only public, private not-for profit and private for-profit.
We definitely want to make these categorical variables. 
We could divide into just public and private, but let's keep further classifications because there are only three

In [ ]:
# Making level and control into categorical variables
college_data_filtered['level'] = college_data_filtered['level'].astype('category')
college_data_filtered['control'] = college_data_filtered['control'].astype('category')

All of our other variables have distinct values. However, we can collapse to narrow down into broad categories.
Specifically, we can divide count of students into levels 1-4, with one being extremely small schools(bottom 25th %), and 4 being very large(top 25%)

In [ ]:
# Boxplot to visualize distribution of size of schools
college_data_filtered.boxplot(column= 'student_count', vert= False, grid=False)

Seems like we have a lot of outliers. May be better to just find the percentiles and assign to variables

In [ ]:
count_pct = np.percentile(college_data_filtered['student_count'], [25, 50, 75])
count_25 =  count_pct[0]
count_50 =  count_pct[1]
count_75 =  count_pct[2]

In [ ]:
# Loop through colleges to assign values based on size, make categorical variable
for index, row in college_data_filtered.iterrows():
    if row['student_count'] < count_pct[0]:
        college_data_filtered.at[index, 'student_count'] = 1
    elif row['student_count'] < count_pct[1]:
        college_data_filtered.at[index, 'student_count'] = 2
    elif row['student_count'] < count_pct[2]:
        college_data_filtered.at[index, 'student_count'] = 3
    else:
        college_data_filtered.at[index, 'student_count'] = 4

college_data_filtered['student_count'] = college_data_filtered['student_count'].astype('category')

Before we scale our numeric variables, we need to define our target variable. Currently we have normalized values for expected award value and awards per value. This represents how common it is for financial awards to be granted and how much they are for, on average. We can multiply these together to create a metric that factors in both. We want to do this before scaling and normalizing to keep our target variable between 0 and 1.

In [ ]:
college_data_filtered['award_total'] = college_data_filtered['awards_per_value'] * college_data_filtered['exp_award_value']


Now we will scale our numeric variables using min-max scaling, placing all variables between 0 and 1 and making sure all features contribute equally to model performance 

In [ ]:
# Makes list of numeric variables, scales them to be between 0 and 1
my_list = list(college_data_filtered.select_dtypes(include=['number']))
college_data_filtered[my_list] = MinMaxScaler().fit_transform(college_data_filtered[my_list])
college_data_filtered

Our numeric variables now clearly are only between 0 and 1. We have successfully scaled and normalized our numeric variables!

The next step will be to implement one-hot encoding on our categorical variables, making them into binary variables

In [ ]:
# Makes a list of categorical variables, sets them to be represented as true or false
category_list = list(college_data_filtered.select_dtypes('category'))
print(category_list)
college_data_filtered_1h = pd.get_dummies(college_data_filtered, columns=category_list)
college_data_filtered_1h



Let's get an idea of what our target variable looks like/represents:

In [ ]:
college_data_filtered_1h.boxplot(column= 'award_total', vert= False, grid=False)
college_data_filtered_1h.award_total.describe()

It seems like a lot of our data is concentrated in lower values. We will define a college with strong funding for awards to be above the 75th percentile, or having an award total metric greater than 0.107

In [ ]:
#Assigns award_total in the top 25% to be represented as a binary variable, basically strong or moderate/weak
college_data_filtered_1h['award_total'] = pd.cut(college_data_filtered_1h.award_total, bins = [-1,0.107236,1], labels =[0,1])
college_data_filtered_1h


In [ ]:
# Checking prevalence of our variable
prevalence = college_data_filtered_1h.award_total.value_counts()[1]/len(college_data_filtered_1h.award_total)
prevalence

This is good as we wanted about 1/4 of colleges to be deemed as having good programs for awarding students money

We can now drop the variables that were used to calculate our 'award_total' variable as they are no longer useful to us. The target variable is just award total. We also do not need name anymore.

In [ ]:
# Drop the columns while keeping the same
college_dt = college_data_filtered_1h.drop(['exp_award_value', 'awards_per_value', 'chronname'], axis=1)
college_dt

Now we can proceed to perform training on our dataset. We will use 25% of the data for training and 75% of the data for testing. 

In [ ]:
num_rows = college_dt.shape[0]
test_num = round(float(num_rows/4), 0)
train_num = num_rows - test_num
print("Total:", num_rows)
print("Training:", train_num)
print("Test:", test_num)

In [ ]:
# partition data, stratify on award_total to make sure we have even distribution of schools with 1's and 0's in both set
Train, Test = train_test_split(college_dt, train_size=2600, stratify=college_dt.award_total, random_state=42)
print(Train.shape)
print(Test.shape)


In [ ]:
# Create tuning set 
Tune, Test = train_test_split(Test,  train_size = .5, stratify= Test.award_total)

In [ ]:
print("Train award total: ", Train.award_total.value_counts())
print(671/(1929+671))

print("Test award total: ", Test.award_total.value_counts())
print(112/(112+322))

print("Tune award total: ", Tune.award_total.value_counts())
print(112/(321+112))

Our distribution looks good. We have correctly assigned our training and testing data.

### Dataset 1 Step 3

#### What do your instincts tell you about the data. Can it address your problem, what areas/items are you worried about? 

Our data can be used to address our problem. Students who want to ensure they will be financially supported for projects and research can find similarities across universities that provide awards at high values and frequencies. They can see whether public or private institutions provide more,  graduation %, and full time student % are correlated with schools that have strong financial award programs. This will allow students to see more baseline and accessible data to determine whether the university will be a good fit for them or not. As for concerns, it could be a potential issue that we were unable to factor in the schools that contained missing data as this might bias our results. However, since it was a small subset of our dataset it is not a great concern.

### Dataset 1 Step 4

#### Function

In [43]:
def college_data_prep(data):
    # Filtering to keep only values we need, dropping NA vals
    college_data_filtered = data[['chronname', 'level', 'control', 'student_count', 'grad_100_percentile',
                                       'ft_pct', 'exp_award_value', 'awards_per_value']]
    college_data_filtered = college_data_filtered.dropna(subset=['grad_100_percentile', 'ft_pct'])
    # Making object variables categorical
    college_data_filtered['level'] = college_data_filtered['level'].astype('category')
    college_data_filtered['control'] = college_data_filtered['control'].astype('category')
    
    # Making student count categorical
    count_pct = np.percentile(college_data_filtered['student_count'], [25, 50, 75])
    count_25 =  count_pct[0]
    count_50 =  count_pct[1]
    count_75 =  count_pct[2]
    for index, row in college_data_filtered.iterrows():
        if row['student_count'] < count_pct[0]:
            college_data_filtered.at[index, 'student_count'] = 1
        elif row['student_count'] < count_pct[1]:
            college_data_filtered.at[index, 'student_count'] = 2
        elif row['student_count'] < count_pct[2]:
            college_data_filtered.at[index, 'student_count'] = 3
        else:
            college_data_filtered.at[index, 'student_count'] = 4
    college_data_filtered['student_count'] = college_data_filtered['student_count'].astype('category')
    # Defining and adding target variable
    college_data_filtered['award_total'] = college_data_filtered['awards_per_value'] * college_data_filtered['exp_award_value']
    # Standardizing numeric data
    my_list = list(college_data_filtered.select_dtypes(include=['number']))
    college_data_filtered[my_list] = MinMaxScaler().fit_transform(college_data_filtered[my_list])
    # One hot encoding for categorical variables
    category_list = list(college_data_filtered.select_dtypes('category'))
    college_data_filtered_1h = pd.get_dummies(college_data_filtered, columns=category_list)
    # Redefines target variable as a 0 or 1
    college_data_filtered_1h['award_total'] = pd.cut(college_data_filtered_1h.award_total, bins = [-1,0.106659,1], labels =[0,1])
    # Drop unneeded columns
    college_dt = college_data_filtered_1h.drop(['exp_award_value', 'awards_per_value', 'chronname'], axis=1)
    #Partition data into train, test, tune
    Train, Test = train_test_split(college_dt, train_size=2600, stratify=college_dt.award_total, random_state=42)
    Tune, Test = train_test_split(Test,  train_size = .5, stratify= Test.award_total)
    return Train.award_total.value_counts(), Test.award_total.value_counts(), Tune.award_total.value_counts(), college_dt

college_data_prep(college_data)

(award_total
 0    1919
 1     681
 Name: count, dtype: int64,
 award_total
 0    320
 1    114
 Name: count, dtype: int64,
 award_total
 0    320
 1    113
 Name: count, dtype: int64,
       grad_100_percentile    ft_pct award_total  level_2-year  level_4-year  \
 0                    0.15  0.935551           0         False          True   
 1                    0.67  0.716216           1         False          True   
 2                    0.00  0.612266           1         False          True   
 3                    0.34  0.733888           0         False          True   
 4                    0.11  0.906445           0         False          True   
 ...                   ...       ...         ...           ...           ...   
 3793                 0.00  0.319127           0         False          True   
 3794                 0.64  0.917879           1         False          True   
 3795                 0.82  0.553015           1         False          True   
 3796          

### Dataset 2 Step 1

In [ ]:
job_data.head()

#### Problems: 
This dataset contains information about job placement, with various metrics as well as a job placement and salary variable. This clearly lends itself to the question of what factors most strongly are correlated with salary. I feel there is a lot that can be explored about the relationship between education and salary from this dataset.

#### Question:
Can we predict salary based on level of education completed as well as performance on exams?

### Dataset 2, Step 2
- The business metric we are looking to analyze is salary. The variables we will be looking at will be degree percentage complete, Secondary Education percentage 10th Grade(test scores from 10th grade), Higher Secondary Education percentage - 12th Grade(test scores from 12th grade), and MBA percentage. We will also look at their field of study/specialization.

### Data Preparation

In [ ]:
# Creating a new dataset that includes only the variables of interest. For now we will keep all education related variables.
job_data_filtered = job_data[['ssc_p', 'hsc_p', 'degree_p', 'salary', 'hsc_s', 'degree_t', 'specialisation']]
job_data_filtered.info()

We want to make sure that all of our data contains our target variable. It is paramount that we address this issue early on as missing values can cause problems with our data.

In [ ]:
print(job_data_filtered.isnull().sum())

The only value that is missing is salary. We can drop these rows because this is our target variable and any data without it is not meaningful to us.

In [ ]:
job_data_filtered = job_data_filtered.dropna()

We have hsc_s, degree_t, and specialisation as object type variables right now but we may want them to be categorical variables.

In [ ]:
# Checking to see how many unique values there are
print("hsc_s classifications:", job_data_filtered['hsc_s'].unique())
print("degree classifications:", job_data_filtered['degree_t'].unique())
print("specialisation classifications:", job_data_filtered['specialisation'].unique())

Since there are only 2-3 unique values for these variables, we can make them into categorical variables.

In [ ]:
job_data_filtered['hsc_s'] = job_data_filtered['hsc_s'].astype('category')
job_data_filtered['degree_t'] = job_data_filtered['degree_t'].astype('category')
job_data_filtered['specialisation'] = job_data_filtered['specialisation'].astype('category')

job_data_filtered.info()

Great. Now we have only numeric and categorical variables. Since there are so few classifications we do not need to collapse variables down further. We can now normalize our numeric data

In [ ]:
# Makes list of numeric variables, scales them to be between 0 and 1
job_num_list = list(job_data_filtered.select_dtypes(include=['number']))
job_data_filtered[job_num_list] = MinMaxScaler().fit_transform(job_data_filtered[job_num_list])
job_data_filtered

Our numeric variables have now clearly been scaled to be between 0 and 1. We have successfully scaled our data. Now we will move to one-hot encoding of our categorical variables.


In [ ]:
# Take our categorical variables and assign them true if present, otherwise false
job_cat_list = list(job_data_filtered.select_dtypes('category'))
job_data_1h = pd.get_dummies(job_data_filtered, columns=job_cat_list)
job_data_1h

Now that we have successfully one-hot encoded the data, we can move to defining our target variable

For this problem, since we are looking at factors such as MBA completion and higher education, we want to see how education correlates with high salary. Let us define a high salary as one that is in the top 30th percentile. 

In [ ]:
job_dt = job_data_1h
job_dt.salary.describe(percentiles=[0.25, 0.50, 0.70, 0.90])

The top 30th percentile is defined as values above 0.1351, so we will represent salary as a binary variable that indicates whether salary is in the top 30th percentile or not

In [ ]:
job_dt['salary'] = pd.cut(job_dt.salary, bins = [-1,0.135,1], labels =[0,1])
job_dt

In [ ]:
# Checking prevalence of our target variable, high salary
sal_prevalence = job_dt.salary.value_counts()[1]/len(job_dt.salary)
sal_prevalence

The prevalence of our target variable represents about 1/3rd of our data. This is acceptable as our top salary being defined as 1/3 of our data makes sense.

In [ ]:
# Calculating values of training and testing data
job_rows = job_dt.shape[0]
job_test = round(float(job_rows/4), 0)
job_train = job_rows - job_test
print("Total:", job_rows)
print("Training:", job_train)
print("Test:", job_test)

In [45]:
# Partitioning data into training and testing, stratifying to ensure top salary data is evenly distributed for both groups
# random state used for repeatability
job_train, job_test = train_test_split(job_dt, train_size=111, stratify=job_dt.salary, random_state=42)
print(job_train.shape)
print(job_test.shape)

(111, 12)
(37, 12)


In [ ]:
# Create tuning set, half of our testing data
job_tune, job_test = train_test_split(job_test, train_size = .5, stratify= job_test.salary)
print(job_tune.shape)
print(job_test.shape)

In [ ]:
print("Train salary total: ", job_train.salary.value_counts())
print(38/(73+38))

print("Test award total: ", job_test.salary.value_counts())
print(6/(13+6))

print("Tune award total: ", job_tune.salary.value_counts())
print(6/(12+6))

We aimed to have a prevalence of about 1/3, and our partitions correctly represent that. The distribution is not as exact as with the college data because we are dealing with a smaller dataset.

Everything looks good. Our function works properly.

### Dataset 2 Step 3

#### What do your instincts tell you about the data. Can it address your problem, what areas/items are you worried about? 


Our dataset gives us a lot of information that can be used to address our problem, especially our numeric variables. We are able to see what test scores, degree completion, and level of education correlate with a high salary. Our categorical variables allow for stratification across fields so we can see how these metrics vary across specialization. We would likely need to build a regression model that shows the correlation between the variables and salary. From this, we could define cutoff points for what test scores and level of education are generally needed to achieve a high salary. This would address our problem as people could use this information to determine how much time and money they need to invest in their education in order for them to have a high salary. A concern of mine would be the dataset is not large enough as we only have slightly over 100 observations. For us to draw more generalizable conclusions, a larger dataset would be preferred. There is great variance in both education and jobs so we want to ensure the general public is well represented. 

### Dataset 2 Step 4
#### Function:

In [44]:
def job_data_prep(data):
    # Selecting our variables of interest
    job_data_filtered = data[['ssc_p', 'hsc_p', 'degree_p', 'salary', 'hsc_s', 'degree_t', 'specialisation']]
    # Making object variables categorical
    job_data_filtered['hsc_s'] = job_data_filtered['hsc_s'].astype('category')
    job_data_filtered['degree_t'] = job_data_filtered['degree_t'].astype('category')
    job_data_filtered['specialisation'] = job_data_filtered['specialisation'].astype('category')
    # Scaling/normalizing our numeric data
    job_num_list = list(job_data_filtered.select_dtypes(include=['number']))
    job_data_filtered[job_num_list] = MinMaxScaler().fit_transform(job_data_filtered[job_num_list])
    # One hot encoding 
    job_cat_list = list(job_data_filtered.select_dtypes('category'))
    job_data_1h = pd.get_dummies(job_data_filtered, columns=job_cat_list)
    # Dropping data that doesn't have salary information
    job_dt = job_data_1h.dropna(subset=['salary'])
    # Making our target variable binary
    job_dt['salary'] = pd.cut(job_dt.salary, bins = [-1,0.135,1], labels =[0,1])
    # Partitioning our data
    job_train, job_test = train_test_split(job_dt, train_size=111, stratify=job_dt.salary, random_state=42)
    job_tune, job_test = train_test_split(job_test, train_size = .5, stratify= job_test.salary)
    return job_train.salary.value_counts(), job_test.salary.value_counts(), job_tune.salary.value_counts(), job_dt

job_data_prep(job_data)

/tmp/ipykernel_3910/4071431339.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  job_data_filtered['hsc_s'] = job_data_filtered['hsc_s'].astype('category')
/tmp/ipykernel_3910/4071431339.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  job_data_filtered['degree_t'] = job_data_filtered['degree_t'].astype('category')
/tmp/ipykernel_3910/4071431339.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

(salary
 0    73
 1    38
 Name: count, dtype: int64,
 salary
 0    13
 1     6
 Name: count, dtype: int64,
 salary
 0    12
 1     6
 Name: count, dtype: int64,
         ssc_p     hsc_p  degree_p salary  hsc_s_Arts  hsc_s_Commerce  \
 0    0.538240  0.889621  0.195122      0       False            True   
 1    0.792414  0.680890  0.670244      0       False           False   
 2    0.497011  0.510708  0.341463      0        True           False   
 4    0.925788  0.602965  0.568293      1       False            True   
 7    0.847454  0.444811  0.390244      0       False           False   
 ..        ...       ...       ...    ...         ...             ...   
 209  0.435168  0.576606  0.365854      0       False            True   
 210  0.818594  0.741351  0.673171      1       False            True   
 211  0.352711  0.378913  0.536585      0       False           False   
 212  0.538240  0.494234  0.560976      0       False            True   
 213  0.682540  0.477759  0.195122 